<a href="https://colab.research.google.com/github/dejunga/ML-Vehicle_Image_Classification/blob/main/train_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
import pickle  # Added to save the history

from google.colab import drive
drive.mount('/content/drive')

# Set up directories for training data
main_dir = "/content/drive/My Drive/Vehicles"  # Adjust this to your path
categories = ["Auto Rickshaws", "Bikes", "Cars", "Motorcycles", "Planes", "Ships", "Trains"]
base_split_dir = "/content/drive/My Drive/vehicles_split"

# Check if the base_split_dir exists, if not create it and split the dataset
if not os.path.exists(base_split_dir):
    split_dirs = ["train", "val", "test"]

    for split_dir in split_dirs:
        for category in categories:
            os.makedirs(os.path.join(base_split_dir, split_dir, category), exist_ok=True)
            print(f"Directory created: {os.path.join(base_split_dir, split_dir, category)}")

    # Split the data and move images to appropriate directories
    for category in categories:
        category_path = os.path.join(main_dir, category)

        # Ensure the directory exists
        if not os.path.exists(category_path):
            print(f"Directory not found: {category_path}")
            continue

        images = os.listdir(category_path)
        train_images, temp_images = train_test_split(images, test_size=0.3, random_state=42)
        val_images, test_images = train_test_split(temp_images, test_size=0.5, random_state=42)

        # Function to move images into respective folders
        def move_images(image_list, destination):
            for image in image_list:
                shutil.copy(os.path.join(category_path, image), os.path.join(base_split_dir, destination, category, image))

        move_images(train_images, "train")
        move_images(val_images, "val")
        move_images(test_images, "test")

    print("Data split into train, validation, and test sets successfully!")

# Check how many images were copied into each category and split
split_dirs = ["train", "val", "test"]
for split_dir in split_dirs:
    for category in categories:
        num_images = len(os.listdir(os.path.join(base_split_dir, split_dir, category)))
        print(f"Found {num_images} images in {split_dir}/{category}")

# Create data generators
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Ensure the directories exist before creating generators
if os.path.exists(os.path.join(base_split_dir, "train")):
    train_generator = train_datagen.flow_from_directory(os.path.join(base_split_dir, "train"), target_size=(150, 150),
                                                        batch_size=32, class_mode='categorical')
else:
    print(f"Training directory not found: {os.path.join(base_split_dir, 'train')}")

if os.path.exists(os.path.join(base_split_dir, "val")):
    val_generator = val_test_datagen.flow_from_directory(os.path.join(base_split_dir, "val"), target_size=(150, 150),
                                                         batch_size=32, class_mode='categorical')
else:
    print(f"Validation directory not found: {os.path.join(base_split_dir, 'val')}")

# Load VGG16 as the base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
base_model.trainable = False

# Build the model
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(categories), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# ***MODIFICATION STARTS HERE***
# Train the model and store the training history
history = None
if os.path.exists(os.path.join(base_split_dir, "train")) and os.path.exists(os.path.join(base_split_dir, "val")):
    history = model.fit(train_generator, epochs=10, validation_data=val_generator)

# Save the training history
history_save_path = '/content/drive/My Drive/models/model_history.pkl'
with open(history_save_path, 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

print(f"Training history saved at {history_save_path}")
# ***MODIFICATION ENDS HERE***

# Save the model in .keras format
save_dir = '/content/drive/My Drive/models'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

model_save_path = os.path.join(save_dir, 'tensorflow_vehicle_model.keras')
model.save(model_save_path)

print(f"Model trained and saved to {model_save_path} successfully!")

Mounted at /content/drive
Found 560 images in train/Auto Rickshaws
Found 560 images in train/Bikes
Found 553 images in train/Cars
Found 560 images in train/Motorcycles
Found 560 images in train/Planes
Found 560 images in train/Ships
Found 560 images in train/Trains
Found 120 images in val/Auto Rickshaws
Found 120 images in val/Bikes
Found 118 images in val/Cars
Found 120 images in val/Motorcycles
Found 120 images in val/Planes
Found 120 images in val/Ships
Found 120 images in val/Trains
Found 120 images in test/Auto Rickshaws
Found 120 images in test/Bikes
Found 119 images in test/Cars
Found 120 images in test/Motorcycles
Found 120 images in test/Planes
Found 120 images in test/Ships
Found 120 images in test/Trains
Found 3911 images belonging to 7 classes.
Found 838 images belonging to 7 classes.
58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/10
 44/123 [=========>....................] - ETA: 10:00 - loss: 1.3198 - accuracy: 0.5987

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


123/123 [==============================] - 1234s 10s/step - loss: 0.8533 - accuracy: 0.7228 - val_loss: 0.3384 - val_accuracy: 0.8819
Epoch 2/10
123/123 [==============================] - 81s 663ms/step - loss: 0.4953 - accuracy: 0.8284 - val_loss: 0.2542 - val_accuracy: 0.9129
Epoch 3/10
123/123 [==============================] - 81s 657ms/step - loss: 0.4396 - accuracy: 0.8387 - val_loss: 0.2539 - val_accuracy: 0.9117
Epoch 4/10
123/123 [==============================] - 82s 669ms/step - loss: 0.4210 - accuracy: 0.8486 - val_loss: 0.2502 - val_accuracy: 0.9117
Epoch 5/10
123/123 [==============================] - 82s 664ms/step - loss: 0.3755 - accuracy: 0.8622 - val_loss: 0.2492 - val_accuracy: 0.9165
Epoch 6/10
123/123 [==============================] - 82s 663ms/step - loss: 0.3833 - accuracy: 0.8614 - val_loss: 0.2376 - val_accuracy: 0.9212
Epoch 7/10
123/123 [==============================] - 82s 666ms/step - loss: 0.3734 - accuracy: 0.8783 - val_loss: 0.2363 - val_accuracy: 0.9